In [15]:
import pandas as pd
from sklearn.preprocessing import RobustScaler
import re
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
import string
import math
from tqdm import tqdm
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('data.csv', index_col = 0)

In [5]:
df

,okapitf,tfidf,okapibm25,ulm,ulmjm,query_len,doc_len,relevance
id,,,,,,,,
54-AP890105-0043,1.072,1.641,4.407,-29.676,-20.472,23.0,176.0,0.0
54-AP890117-0119,2.192,3.312,8.147,-28.229,-16.762,23.0,256.0,1.0
54-AP890118-0061,2.178,3.265,8.078,-28.375,-16.825,23.0,232.0,1.0
54-AP890118-0257,1.550,1.943,5.566,-29.172,-17.634,23.0,287.0,0.0
54-AP890121-0007,0.601,0.813,2.129,-30.075,-23.267,23.0,245.0,0.0
...,...,...,...,...,...,...,...,...
99-AP890921-0266,0.373,0.321,0.943,-20.150,-15.041,9.0,182.0,0.0
99-AP890921-0267,0.000,0.000,0.000,-20.449,-16.793,9.0,56.0,0.0
99-AP890921-0268,0.000,0.000,0.000,-20.450,-16.793,9.0,66.0,0.0


In [6]:

df['relevance'] = df['relevance'].astype(int)



In [7]:
df.head()

,okapitf,tfidf,okapibm25,ulm,ulmjm,query_len,doc_len,relevance
id,,,,,,,,
54-AP890105-0043,1.072,1.641,4.407,-29.676,-20.472,23.0,176.0,0
54-AP890117-0119,2.192,3.312,8.147,-28.229,-16.762,23.0,256.0,1
54-AP890118-0061,2.178,3.265,8.078,-28.375,-16.825,23.0,232.0,1
54-AP890118-0257,1.550,1.943,5.566,-29.172,-17.634,23.0,287.0,0
54-AP890121-0007,0.601,0.813,2.129,-30.075,-23.267,23.0,245.0,0


In [10]:
rs = RobustScaler()
columns = ['okapitf', 'tfidf', 'okapibm25', 'ulm', 'ulmjm', 'query_len', 'doc_len']
df[columns] = rs.fit_transform(df[columns])

In [11]:
df

,okapitf,tfidf,okapibm25,ulm,ulmjm,query_len,doc_len,relevance
id,,,,,,,,
54-AP890105-0043,1.867596,5.451827,1.579428,-0.931729,-0.270038,1.285714,-0.386139,0
54-AP890117-0119,3.818815,11.003322,2.919810,-0.790200,0.002275,1.285714,0.009901,1
54-AP890118-0061,3.794425,10.847176,2.895081,-0.804480,-0.002349,1.285714,-0.108911,1
54-AP890118-0257,2.700348,6.455150,1.994803,-0.882433,-0.061729,1.285714,0.163366,0
54-AP890121-0007,1.047038,2.700997,0.763014,-0.970755,-0.475191,1.285714,-0.044554,0
...,...,...,...,...,...,...,...,...
99-AP890921-0266,0.649826,1.066445,0.337963,0.000000,0.128597,-0.714286,-0.356436,0
99-AP890921-0267,0.000000,0.000000,0.000000,-0.029245,0.000000,-0.714286,-0.980198,0
99-AP890921-0268,0.000000,0.000000,0.000000,-0.029343,0.000000,-0.714286,-0.930693,0


In [16]:
# Function to read queries
def read_queries_from_file(file_path):
    queries = {}

    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(r'(\d+)\.\s+(.*)', line)
            if match:
                number = match.group(1)
                query_text = match.group(2).strip()  
                queries[number] = query_text

    return queries

In [17]:
# Extracting queries 
file_path = 'query_desc.51-100.short.txt'
queries = read_queries_from_file(file_path)
queries

{'85': 'Document will discuss allegations, or measures being taken against, corrupt public officials of any governmental jurisdiction worldwide.',
 '59': 'Document will report a type of weather event which has directly caused at least one fatality in some location.',
 '56': 'Document will include a prediction about the prime lending rate, or will report an actual prime rate move.',
 '71': 'Document will report incursions by land, air, or water into the border area of one country by military forces of a second country or a guerrilla group based in a second country.',
 '64': 'Document will report an event or result of politically motivated hostage-taking.',
 '62': "Document will report a military coup d'etat, either attempted or successful, in any country.",
 '93': 'Document must describe or identify supporters of the National Rifle Association (NRA), or its assets.',
 '99': 'Document will identify a development in the Iran-Contra Affair.',
 '58': 'Document will predict or anticipate a r

In [23]:
import random


def query_split(query_ids, train_size=20, test_size=5):
    
    test_ids = random.sample(query_ids, test_size)
    
    
    train_ids = [qid for qid in query_ids if qid not in test_ids]
    
    
    if len(train_ids) > train_size:
        train_ids = random.sample(train_ids, train_size)
    
    return train_ids, test_ids



In [27]:
train_queries, test_queries = query_split(list(queries.keys()))

print(train_queries)
print(test_queries)

['85', '59', '56', '71', '64', '93', '99', '58', '77', '54', '87', '100', '61', '95', '68', '97', '98', '60', '63', '91']
['80', '62', '89', '94', '57']


In [28]:
train_set = pd.DataFrame()
test_set = pd.DataFrame()

In [35]:
def get_training_data(df, train_ids):
    training_data = df[df.index.astype(str).str.split('-').str[0].isin(train_ids)]
    return training_data
        

In [36]:
def get_testing_data(df, test_ids):
    
    testing_data = df[df.index.astype(str).str.split('-').str[0].isin(test_ids)]
    
    return testing_data


In [40]:
train_set = get_training_data(df, train_queries)
test_set = get_testing_data(df, test_queries)

In [42]:
train_set

,okapitf,tfidf,okapibm25,ulm,ulmjm,query_len,doc_len,relevance
id,,,,,,,,
54-AP890105-0043,1.867596,5.451827,1.579428,-0.931729,-0.270038,1.285714,-0.386139,0
54-AP890117-0119,3.818815,11.003322,2.919810,-0.790200,0.002275,1.285714,0.009901,1
54-AP890118-0061,3.794425,10.847176,2.895081,-0.804480,-0.002349,1.285714,-0.108911,1
54-AP890118-0257,2.700348,6.455150,1.994803,-0.882433,-0.061729,1.285714,0.163366,0
54-AP890121-0007,1.047038,2.700997,0.763014,-0.970755,-0.475191,1.285714,-0.044554,0
...,...,...,...,...,...,...,...,...
99-AP890921-0266,0.649826,1.066445,0.337963,0.000000,0.128597,-0.714286,-0.356436,0
99-AP890921-0267,0.000000,0.000000,0.000000,-0.029245,0.000000,-0.714286,-0.980198,0
99-AP890921-0268,0.000000,0.000000,0.000000,-0.029343,0.000000,-0.714286,-0.930693,0


In [43]:
x_train, y_train, x_test, y_test = train_set.iloc[:, :-1], train_set.iloc[:, -1], test_set.iloc[:,:-1], test_set.iloc[:, -1]

In [44]:
x_train

,okapitf,tfidf,okapibm25,ulm,ulmjm,query_len,doc_len
id,,,,,,,
54-AP890105-0043,1.867596,5.451827,1.579428,-0.931729,-0.270038,1.285714,-0.386139
54-AP890117-0119,3.818815,11.003322,2.919810,-0.790200,0.002275,1.285714,0.009901
54-AP890118-0061,3.794425,10.847176,2.895081,-0.804480,-0.002349,1.285714,-0.108911
54-AP890118-0257,2.700348,6.455150,1.994803,-0.882433,-0.061729,1.285714,0.163366
54-AP890121-0007,1.047038,2.700997,0.763014,-0.970755,-0.475191,1.285714,-0.044554
...,...,...,...,...,...,...,...
99-AP890921-0266,0.649826,1.066445,0.337963,0.000000,0.128597,-0.714286,-0.356436
99-AP890921-0267,0.000000,0.000000,0.000000,-0.029245,0.000000,-0.714286,-0.980198
99-AP890921-0268,0.000000,0.000000,0.000000,-0.029343,0.000000,-0.714286,-0.930693


In [45]:
y_train

id
54-AP890105-0043    0
54-AP890117-0119    1
54-AP890118-0061    1
54-AP890118-0257    0
54-AP890121-0007    0
                   ..
99-AP890921-0266    0
99-AP890921-0267    0
99-AP890921-0268    0
99-AP890921-0269    0
99-AP890921-0270    0
Name: relevance, Length: 21631, dtype: int64

In [46]:
x_test

,okapitf,tfidf,okapibm25,ulm,ulmjm,query_len,doc_len
id,,,,,,,
57-AP890126-0252,1.792683,5.551495,3.703969,0.558979,0.743541,-0.142857,0.014851
57-AP890126-0273,1.926829,6.289037,4.045874,0.605536,0.738917,-0.142857,1.074257
57-AP890127-0038,0.752613,2.714286,0.833259,0.517214,0.377202,-0.142857,0.762376
57-AP890130-0258,0.932056,3.358804,0.979124,0.529441,0.390414,-0.142857,0.693069
57-AP890201-0272,1.501742,8.544850,2.144611,0.576193,0.513726,-0.142857,-0.163366
...,...,...,...,...,...,...,...
94-AP890921-0045,0.000000,0.000000,0.000000,0.470657,0.319289,-0.285714,-1.163366
94-AP890921-0046,0.000000,0.000000,0.000000,0.470755,0.319289,-0.285714,-0.732673
94-AP890921-0047,0.000000,0.000000,0.000000,0.470559,0.319289,-0.285714,0.722772


## Logistic Regression

In [47]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter= 1000, solver='liblinear', C=0.01, penalty='l1')
lr.fit(x_train, y_train)

LogisticRegression(C=0.01, max_iter=1000, penalty='l1', solver='liblinear')

In [48]:
result = lr.predict(x_test)

In [49]:
result

array([0, 1, 0, ..., 0, 0, 0])

### Testing the model - test data

In [50]:
result = lr.predict_proba(x_test)

In [51]:
result

array([[0.59236089, 0.40763911],
       [0.48195331, 0.51804669],
       [0.86318248, 0.13681752],
       ...,
       [0.95728171, 0.04271829],
       [0.96153084, 0.03846916],
       [0.96329214, 0.03670786]])

In [53]:
result = lr.predict_proba(x_test)[:, 1]
print(result)

[0.40763911 0.51804669 0.13681752 ... 0.04271829 0.03846916 0.03670786]


In [68]:
# def write_predictions_to_text_file(test_set, predictions, file_path):
#     with open(file_path, 'w') as file:
#         i = 0
        
#         for (index, prediction) in zip(test_set.index, predictions):
#             query_id, document_id = index.split('-',1)[:2]
            
#             file.write(f"{query_id} Q0 {document_id} 1 {prediction} exp\n")


In [94]:
def write_predictions_to_text_file(test_set, predictions, file_path):
    
    
    results_df = pd.DataFrame({
        'index': test_set.index,
        'prediction': predictions
    })

    
    results_df[['query_id', 'document_id']] = results_df['index'].str.split('-', n = 1, expand=True)

    
    results_df.sort_values(by=['query_id', 'prediction'], ascending=[True, False], inplace=True)

    
    results_df['rank'] = results_df.groupby('query_id').cumcount() + 1

    with open(file_path, 'w') as file:
        for _, row in results_df.iterrows():
            file.write(f"{row['query_id']} Q0 {row['document_id']} {row['rank']} {row['prediction']} exp\n")



In [88]:
write_predictions_to_text_file(test_set, result, "result1.txt")

In [98]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result1.txt 


Queryid (Num):       57
Total number of documents over all queries
    Retrieved:     1000
    Relevant:        18
    Rel_ret:         18
Interpolated Recall - Precision Averages:
    at 0.00       1.0000
    at 0.10       1.0000
    at 0.20       0.8000
    at 0.30       0.6316
    at 0.40       0.6316
    at 0.50       0.6316
    at 0.60       0.6316
    at 0.70       0.6190
    at 0.80       0.5769
    at 0.90       0.4390
    at 1.00       0.4390
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.6484
Precision:
  At    5 docs:   0.8000
  At   10 docs:   0.6000
  At   15 docs:   0.6000
  At   20 docs:   0.6000
  At   30 docs:   0.5000
  At  100 docs:   0.1800
  At  200 docs:   0.0900
  At  500 docs:   0.0360
  At 1000 docs:   0.0180
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.6111

Queryid (Num):       62
Total number of documents over all queries
    Retrieved:     1000
    Relevant:   

In [90]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result1.txt > testingperformance.txt

### Testing the model - Train Data

In [95]:
result_train = lr.predict_proba(x_train)[:, 1]
result_train

array([0.15033291, 0.77242244, 0.75768987, ..., 0.02346248, 0.02389444,
       0.02709578])

In [96]:
write_predictions_to_text_file(train_set, result_train, "result-train.txt")

In [97]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-train.txt 


Queryid (Num):      100
Total number of documents over all queries
    Retrieved:     1000
    Relevant:        31
    Rel_ret:         31
Interpolated Recall - Precision Averages:
    at 0.00       1.0000
    at 0.10       1.0000
    at 0.20       1.0000
    at 0.30       0.8333
    at 0.40       0.5909
    at 0.50       0.5000
    at 0.60       0.4130
    at 0.70       0.3385
    at 0.80       0.3049
    at 0.90       0.1892
    at 1.00       0.0824
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.5637
Precision:
  At    5 docs:   1.0000
  At   10 docs:   0.8000
  At   15 docs:   0.7333
  At   20 docs:   0.6000
  At   30 docs:   0.5000
  At  100 docs:   0.2600
  At  200 docs:   0.1450
  At  500 docs:   0.0620
  At 1000 docs:   0.0310
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.4839

Queryid (Num):       54
Total number of documents over all queries
    Retrieved:     1000
    Relevant:   

In [99]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-train.txt > trainingperformance.txt

In [73]:
def query_split(query_ids, train_size=20, test_size=5, previously_tested_ids=None):
    
    if previously_tested_ids is not None:
        available_for_testing = [qid for qid in query_ids if qid not in previously_tested_ids]
    else:
        available_for_testing = query_ids
    

    test_ids = random.sample(available_for_testing, test_size)
    
    train_ids = [qid for qid in query_ids if qid not in test_ids]
    
    if len(train_ids) > train_size:
        train_ids = random.sample(train_ids, train_size)
    
    return train_ids, test_ids

In [100]:
tested_queries = []

In [104]:
for i in range(5):
    if len(tested_queries) == 0:
        train_queries, test_queries = query_split(list(queries.keys()))
    else:
        train_queries, test_queries = query_split(list(queries.keys()), 20, 5, tested_queries)
    
    train_set = get_training_data(df, train_queries)
    test_set = get_testing_data(df, test_queries)
    x_train, y_train, x_test, y_test = train_set.iloc[:, :-1], train_set.iloc[:, -1], test_set.iloc[:,:-1], test_set.iloc[:, -1]
    lr = LogisticRegression(max_iter= 1000, solver='liblinear', C=0.01, penalty='l1')
    lr.fit(x_train, y_train)    
    result = lr.predict_proba(x_test)[:, 1]
    write_predictions_to_text_file(test_set, result, "result-test" + str(i+1) + ".txt")
    tested_queries.extend(test_queries)

In [106]:
len(tested_queries)

25

In [107]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test1.txt > testingperformance-test1.txt

In [108]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test2.txt > testingperformance-test2.txt

In [109]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test3.txt > testingperformance-test3.txt

In [110]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test4.txt > testingperformance-test4.txt

In [111]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test5.txt > testingperformance-test5.txt

In [113]:
train_queries, test_queries = query_split(list(queries.keys()))
train_set = get_training_data(df, train_queries)
test_set = get_testing_data(df, test_queries)
x_train, y_train, x_test, y_test = train_set.iloc[:, :-1], train_set.iloc[:, -1], test_set.iloc[:,:-1], test_set.iloc[:, -1]
lr = LogisticRegression(max_iter= 1000, solver='liblinear', C=0.01, penalty='l1')
lr.fit(x_train, y_train)    

LogisticRegression(C=0.01, max_iter=1000, penalty='l1', solver='liblinear')

**Testing - Metrics**

In [122]:
from sklearn.metrics import accuracy_score, classification_report 
y_pred = lr.predict(x_test)
    
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {round(accuracy,3)}")


Accuracy: 0.876


In [123]:
print(report)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      5000
           1       0.25      0.18      0.21       497

    accuracy                           0.88      5497
   macro avg       0.58      0.56      0.57      5497
weighted avg       0.86      0.88      0.87      5497



**Training - Metrics**

In [124]:
from sklearn.metrics import accuracy_score, classification_report 
y_pred = lr.predict(x_train)
    
accuracy = accuracy_score(y_train, y_pred)
report = classification_report(y_train, y_pred)
print(f"Accuracy: {round(accuracy,3)}")


Accuracy: 0.947


In [126]:
print(report)

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     20000
           1       0.68      0.30      0.42      1335

    accuracy                           0.95     21335
   macro avg       0.82      0.65      0.70     21335
weighted avg       0.94      0.95      0.94     21335



## SVM

In [128]:
from sklearn import svm

train_queries, test_queries = query_split(list(queries.keys()))
train_set = get_training_data(df, train_queries)
test_set = get_testing_data(df, test_queries)
x_train, y_train, x_test, y_test = train_set.iloc[:, :-1], train_set.iloc[:, -1], test_set.iloc[:,:-1], test_set.iloc[:, -1]
sv = svm.SVC(C = 1, gamma = 0.4, kernel = 'rbf', probability=True)
sv.fit(x_train, y_train)    

SVC(C=1, gamma=0.4, probability=True)

### Testing the model - test data

In [130]:
result = sv.predict_proba(x_test)[:, 1]
write_predictions_to_text_file(test_set, result, "result-test-svm.txt")
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test-svm.txt


Queryid (Num):       56
Total number of documents over all queries
    Retrieved:     1000
    Relevant:       167
    Rel_ret:        166
Interpolated Recall - Precision Averages:
    at 0.00       1.0000
    at 0.10       0.9565
    at 0.20       0.9481
    at 0.30       0.9481
    at 0.40       0.9481
    at 0.50       0.9397
    at 0.60       0.9397
    at 0.70       0.9379
    at 0.80       0.9379
    at 0.90       0.9042
    at 1.00       0.0000
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.9172
Precision:
  At    5 docs:   1.0000
  At   10 docs:   0.9000
  At   15 docs:   0.9333
  At   20 docs:   0.9500
  At   30 docs:   0.9000
  At  100 docs:   0.9300
  At  200 docs:   0.8100
  At  500 docs:   0.3320
  At 1000 docs:   0.1660
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.9042

Queryid (Num):       60
Total number of documents over all queries
    Retrieved:     1000
    Relevant:   

In [131]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-test-svm.txt > testingperformance-svm.txt

### Testing the model - train data

In [132]:
result = sv.predict_proba(x_train)[:, 1]
write_predictions_to_text_file(train_set, result, "result-train-svm.txt")
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-train-svm.txt


Queryid (Num):      100
Total number of documents over all queries
    Retrieved:     1000
    Relevant:        31
    Rel_ret:         31
Interpolated Recall - Precision Averages:
    at 0.00       1.0000
    at 0.10       1.0000
    at 0.20       1.0000
    at 0.30       1.0000
    at 0.40       0.9444
    at 0.50       0.9444
    at 0.60       0.8636
    at 0.70       0.8148
    at 0.80       0.7879
    at 0.90       0.6744
    at 1.00       0.0381
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.8450
Precision:
  At    5 docs:   1.0000
  At   10 docs:   1.0000
  At   15 docs:   0.9333
  At   20 docs:   0.8500
  At   30 docs:   0.7667
  At  100 docs:   0.2900
  At  200 docs:   0.1450
  At  500 docs:   0.0600
  At 1000 docs:   0.0310
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.7742

Queryid (Num):       54
Total number of documents over all queries
    Retrieved:     1000
    Relevant:   


Queryid (Num):       97
Total number of documents over all queries
    Retrieved:     1000
    Relevant:        10
    Rel_ret:         10
Interpolated Recall - Precision Averages:
    at 0.00       1.0000
    at 0.10       1.0000
    at 0.20       1.0000
    at 0.30       1.0000
    at 0.40       1.0000
    at 0.50       1.0000
    at 0.60       1.0000
    at 0.70       1.0000
    at 0.80       0.9000
    at 0.90       0.9000
    at 1.00       0.7143
Average precision (non-interpolated) for all rel docs(averaged over queries)
                  0.9503
Precision:
  At    5 docs:   1.0000
  At   10 docs:   0.9000
  At   15 docs:   0.6667
  At   20 docs:   0.5000
  At   30 docs:   0.3333
  At  100 docs:   0.1000
  At  200 docs:   0.0500
  At  500 docs:   0.0200
  At 1000 docs:   0.0100
R-Precision (precision after R (= num_rel for a query) docs retrieved):
    Exact:        0.9000

Queryid (Num):       98
Total number of documents over all queries
    Retrieved:     1000
    Relevant:   

In [133]:
!perl trec_eval.pl -q qrels.adhoc.51-100.AP89.txt result-train-svm.txt > trainingperformance-svm.txt

**Testing - Metrics**

In [135]:
y_pred = sv.predict(x_test)
    
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {round(accuracy,3)}")

Accuracy: 0.907


In [137]:
print(report)

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5000
           1       0.67      0.34      0.45       634

    accuracy                           0.91      5634
   macro avg       0.80      0.66      0.70      5634
weighted avg       0.89      0.91      0.89      5634



**Training - Metrics**

In [139]:
y_pred = sv.predict(x_train)
    
accuracy = accuracy_score(y_train, y_pred)
report = classification_report(y_train, y_pred)
print(f"Accuracy: {round(accuracy,3)}")

Accuracy: 0.957


In [140]:
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     20000
           1       0.81      0.31      0.45      1198

    accuracy                           0.96     21198
   macro avg       0.88      0.65      0.71     21198
weighted avg       0.95      0.96      0.95     21198

